<a href="https://colab.research.google.com/github/StillWork/AIDD-LAIDD/blob/main/z4_DeepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 화합물 특성 예측 (독성 예측)

- MLP(Multi Layer Perceptron)모델 사용

## 신경망의 특징
- 60년대부터 가능성을 연구하였으나 오랫동안 성과를 내지 못했다
 - 학습이 어렵고, 계산량이 많고, 과대적합이 많고, 블랙박스 동작
 - 지금은 이러한 문제들이, 데이터의 증가, 알고리즘의 발달(공개 SW), 하드웨어 계산능력의 증대로 해결되었다
- small 데이터에 대해서는 전통적인 머신러닝 모델 (RF, SVM 등)을 사용해도 과대적합도 적게 발생하고 잘 동작했다
- 2012년 Imagenet 경진대회에서 CNN 기반의 신경망이 매우 우수한 성적을 내면서 딥러닝 모델이 확산되었다
- 자연어 처리분야에서는 2014년 word2vec(단어 임베딩)의 도입으로 성능이 크게 향상되었다
 - 비슷한 의미를 갖는 단어는 비슷한 표현형을 갖게 함
 - 자연어 처리에서는 초기에는 RNN을 사용했으나 현재는 Transformer를 사용한다

## 신약개발에 신경망 도입
- 2012년 Merck Molecular Activity Challenge 대회에서 신경망 모델이 RF 모델보다 월등히 높은 성능을 냈다
 - 15개의 태스크에 대해 바이오 활동성을 예측하는 문제
 - 이후 QSAR, 분자의 특성 예측, ADME 예측 등 신약개발에 신경망이 적극 도입되기 시작했다

# import

In [ ]:
!pip install deepchem

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 608 kB 32.4 MB/s 
     |████████████████████████████████| 29.1 MB 1.4 MB/s 


In [1]:
import deepchem as dc
import numpy as np
import matplotlib.pyplot as plt
import keras
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/home/dilab162/anaconda3/lib/python3.9/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'pytorch_lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [2]:
from keras.datasets import mnist
from keras.utils.np_utils import to_categorical
from keras import layers
from keras import models

# MLP
- Multi Layer Perceptron

<img src="https://github.com/StillWork/image/blob/main/%E1%84%89%E1%85%B3%E1%84%8F%E1%85%B3%E1%84%85%E1%85%B5%E1%86%AB%E1%84%89%E1%85%A3%E1%86%BA%202022-11-21%20%E1%84%8B%E1%85%A9%E1%84%92%E1%85%AE%2011.48.10.png?raw=1" width="600"  align="left">

## Fringerprint를 사용한 독성 예측 예

- MLP 모델을 사용 (한개의 히든 계층 사용)
- 12개의 이진 분류 태스크를 수행
- auc와 accuracy를 측정

In [3]:
tasks, datasets, transformers = dc.molnet.load_tox21(featurizer='ECFP')
train_dataset, valid_dataset, test_dataset = datasets
print(train_dataset)

<DiskDataset X.shape: (6264, 1024), y.shape: (6264, 12), w.shape: (6264, 12), task_names: ['NR-AR' 'NR-AR-LBD' 'NR-AhR' ... 'SR-HSE' 'SR-MMP' 'SR-p53']>


### Task (표적 단백질)

- Task는 아래 12가지 표적 단백질로 구성된다
- 표적 단백질은 잠재적으로 신약에 활용될 수 있는 분자와 결합시 독성 반응을 보이는 것으로 여겨진다

In [4]:
tasks

['NR-AR',
 'NR-AR-LBD',
 'NR-AhR',
 'NR-Aromatase',
 'NR-ER',
 'NR-ER-LBD',
 'NR-PPAR-gamma',
 'SR-ARE',
 'SR-ATAD5',
 'SR-HSE',
 'SR-MMP',
 'SR-p53']

### 데이터 확인

- tox21_datasets는 train, valid, test의 3가지 데이터셋으로 구성되어있다
- 각 데이터셋에는 X, y, w 벡터가 존재하며 .shape 명령으로 구조를 확인 가능
- X벡터 = 학습 또는 추론에 사용할 feature(특징), 각 샘플은 분자들을 의미
- y벡터 = 학습 또는 추론결과인 정답, 참값등, 각 샘플의 레이블 12개는 표적 단백질 12종과의 결합 정도를 의미
- w벡터 = 가중치(weight)값

In [5]:
train_dataset.X.shape

(6264, 1024)

In [6]:
train_dataset.y.shape

(6264, 12)

In [7]:
train_dataset.w

array([[1.04502242, 1.03632599, 1.12502653, ..., 1.05576503, 1.17464996,
        1.05288369],
       [1.04502242, 1.03632599, 1.12502653, ..., 1.05576503, 1.17464996,
        1.05288369],
       [1.04502242, 1.03632599, 1.12502653, ..., 1.05576503, 0.        ,
        1.05288369],
       ...,
       [1.04502242, 0.        , 1.12502653, ..., 1.05576503, 6.7257384 ,
        1.05288369],
       [1.04502242, 1.03632599, 1.12502653, ..., 1.05576503, 6.7257384 ,
        1.05288369],
       [1.04502242, 1.03632599, 1.12502653, ..., 0.        , 1.17464996,
        1.05288369]])

- `y`의 모양이 (6264, 12)이며 이는 12가지의 멀티태스크가 있음을 나타낸다
- w (가중치)의 의미
 - 이 값이 0이면 결측치를 나타낸다. 해당 샘플에 대해서는 손실함수나 성능평가 시에 무시하도록 한다
 - 이 값은 대부분의 경우 1 또는 1 근처의 값을 갖는다
 - 이 값은 각 태스크별로 레이블 분포의 불균형을 보완하기 위해서 사용된다
 - 12개 태스크가 균등하게 성능에 기여하도록 조정하는데 사용된다.


### 불완전한 데이터세트의 적용(가중치를 통한 걸러내기)

- Tox21 데이터셋은 모든 분자와 단백질간의 생화학적 분석이 완료되어있지 않아 결측치 존재
- 모델 학습시 이러한 결측치는 제외되어야 함
- 각 분자 샘플의 가중치를 기록하는 w벡터를 활용하여 해결
- w벡터에는 손실함수를 계산시 샘플과 task에 곱하여 합산하는 가중치값이 포함
- 결측치가 있는 샘플의 경우, 가중치값이 0이므로 손실함수에 대한 영향없이 무시  
(tox21데이터는 지속적으로 갱신되므로 변할 수 있음)

In [8]:
# np.count_nonzero()를 통해 0인 가중치가 얼마나 있는지 확인하기
print(train_dataset.w.shape)
print(np.count_nonzero(train_dataset.w))
print(np.count_nonzero(train_dataset.w==0))

(6264, 12)
63647
11521


### 데이터 사용을 위한 Transformer(변환기)

- transformers 객체는 원본 데이터셋을 변환시켜주는 tool을 포함
- tox21의 분자 대부분은 표적 단백질에 결합하지 않는 데이터임
- 따라서 y벡터의 레이블 대부분이(약 90%) 결합되지 않음을 나타내는 0으로 채워짐
- 항상 결과를 0으로만 예측하는 모델은 정확도가 90%로 측정될 수 있음
- 이렇게 불균형한 데이터를 보완할 수 있도록 가중치 행렬을 조정해주는 Balancing transformer 사용
- 각 클래스(분류 목표)에 할당된 총 가중치가 동일하도록 개별 데이터들의 가중치를 조정

In [9]:
transformers

## 학습 모델 불러오기

- 학습 모델로 사용하는 MultitaskClassifier는 다중작업분류기임
- 모든 샘플(x)에 대해 여러개의 레이블(y값은 12개)이 있는 다중 분류 문제를 해결하는데 사용
- n_tasks = 분류해야할 작업의 갯수 = 12개  
- n_features = 입력될 feature의 갯수 = x벡터 크기인 (6264x1024)를 따라 1024로 설정  
- layer_sizes = hidden레이어의 갯수(1)와 너비(1000)  

In [19]:
model = dc.models.MultitaskClassifier(n_tasks=12, n_features=1024, layer_sizes=[1000])

## 모델 학습

- 대부분의 머신러닝 라이브러리들은 모델 학습을 함수화한 fit 함수를 지원
- 위에서 지정한 모델에 train_dataset을 사용하여 15 epoch 학습함
- Epoch란, 신경망 모델에 대하여 전체 데이터셋을 모두 사용하여 순전파, 역전파를 모두 진행하고 학습과정을 완료하였다는 의미

In [23]:
model.fit(train_dataset, nb_epoch=15)

0.20108428001403808

## 모델의 성능 평가

- 학습을 모두 마친 모델은 성능 평가 과정을 거쳐야 함
- 성능평가를 위하여 평가 지표(Metric)을 설정
- 본 예제는 분류 문제이므로, ROC_AUC 점수를 평가지표로 사용하되, 다중 분류이므로 각 분류 점수의 평균값을 사용

In [24]:
metric = dc.metrics.Metric(dc.metrics.roc_auc_score, np.mean)

- 위 fit과 같이 evaluate를 통해 학습된 모델에 대해 평가를 진행하되, 이미 학습했던 데이터는 사용하지 않아야 함
- 미리 분리해둔 test 데이터를 활용하면 객관적인 모델 평가가 가능
- train데이터와 test데이터 모두 점수가 높다면 모델이 일반화(Generalization)가 잘 된것으로 평가
- 일반화(Generalization) =  모델이 학습한 데이터 외에도 새로운 데이터에 대한 추론 성능이 높음

In [25]:
metric1 = dc.metrics.Metric(dc.metrics.roc_auc_score)
metric2 = dc.metrics.Metric(dc.metrics.accuracy_score)
print('training set score:', model.evaluate(train_dataset, [metric1, metric2], transformers))
print('test set score:', model.evaluate(test_dataset, [metric1, metric2], transformers))

training set score: {'roc_auc_score': 0.9863932239204063, 'accuracy_score': 0.9450564069816944}
test set score: {'roc_auc_score': 0.6681084934075353, 'accuracy_score': 0.8560799319727893}
